## Hypothesis Testing Assignment

In [265]:
# importing necessary packages
import pandas as pd
import numpy as np
import datetime as dt
# package with hypothesis tests
import scipy.stats as st

### Data

You can download the data from [**here**](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg). The data contains results of all NBA games from seasons 2013/2014 to 2015/2016.

In [266]:
data = pd.read_csv("data/nba_games_2013_2015.csv", delimiter=";")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7380 entries, 0 to 7379
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          7380 non-null   int64  
 1   TEAM_ID            7380 non-null   int64  
 2   TEAM_ABBREVIATION  7380 non-null   object 
 3   TEAM_NAME          7380 non-null   object 
 4   GAME_ID            7380 non-null   int64  
 5   GAME_DATE          7380 non-null   object 
 6   MATCHUP            7380 non-null   object 
 7   WL                 7380 non-null   object 
 8   MIN                7380 non-null   int64  
 9   PTS                7380 non-null   int64  
 10  FGM                7380 non-null   int64  
 11  FGA                7380 non-null   int64  
 12  FG_PCT             7380 non-null   float64
 13  FG3M               7380 non-null   int64  
 14  FG3A               7380 non-null   int64  
 15  FG3_PCT            7380 non-null   float64
 16  FTM                7380 

In [267]:
data["SEASON_ID"].unique()

array([22015, 22014, 22013])

In [268]:
data["GAME_DATE"].unique()

array(['2016-04-13', '2016-04-12', '2016-04-11', '2016-04-10',
       '2016-04-09', '2016-04-08', '2016-04-07', '2016-04-06',
       '2016-04-05', '2016-04-03', '2016-04-02', '2016-04-01',
       '2016-03-31', '2016-03-30', '2016-03-29', '2016-03-28',
       '2016-03-27', '2016-03-26', '2016-03-25', '2016-03-24',
       '2016-03-23', '2016-03-22', '2016-03-21', '2016-03-20',
       '2016-03-19', '2016-03-18', '2016-03-17', '2016-03-16',
       '2016-03-15', '2016-03-14', '2016-03-13', '2016-03-12',
       '2016-03-11', '2016-03-10', '2016-03-09', '2016-03-08',
       '2016-03-07', '2016-03-06', '2016-03-05', '2016-03-04',
       '2016-03-03', '2016-03-02', '2016-03-01', '2016-02-29',
       '2016-02-28', '2016-02-27', '2016-02-26', '2016-02-25',
       '2016-02-24', '2016-02-23', '2016-02-22', '2016-02-21',
       '2016-02-20', '2016-02-19', '2016-02-18', '2016-02-11',
       '2016-02-10', '2016-02-09', '2016-02-08', '2016-02-07',
       '2016-02-06', '2016-02-05', '2016-02-04', '2016-

--------------
### Task 1
Split the data into **3** separate dataframes for each NBA season!

In [269]:
data_2013 = data[data["SEASON_ID"] == 22013]
data_2014 = data[data["SEASON_ID"] == 22014]
data_2015 = data[data["SEASON_ID"] == 22015]

In [270]:
print(f"There are {len(data_2013)} games recorded in the 2013/2014 season.")
print(f"There are {len(data_2014)} games recorded in the 2014/2015 season.")
print(f"There are {len(data_2015)} games recorded in the 2015/2016 season.")

There are 2460 games recorded in the 2013/2014 season.
There are 2460 games recorded in the 2014/2015 season.
There are 2460 games recorded in the 2015/2016 season.


---------------
### Task 2
Test the hypothesis that the offensive productions stats of the Cleveland Cavaliers and Golden State Warriors (the teams that met in the finals that year) were from the same distribution in the 2015/2016 season.

Offensive production refers to two variables: **PTS (Points)** and **FG_PCT (Field Goal Percentage)**. We will need to do two separate hypothesis tests, one for each variable.

#### Team Data

In [271]:
data_2015["TEAM_ID"].unique()

array([1610612750, 1610612749, 1610612738, 1610612747, 1610612739,
       1610612758, 1610612746, 1610612764, 1610612737, 1610612761,
       1610612740, 1610612753, 1610612762, 1610612744, 1610612741,
       1610612748, 1610612763, 1610612759, 1610612766, 1610612751,
       1610612742, 1610612745, 1610612755, 1610612757, 1610612756,
       1610612743, 1610612754, 1610612765, 1610612752, 1610612760])

In [272]:
data_2015["TEAM_NAME"].unique()

array(['Minnesota Timberwolves', 'Milwaukee Bucks', 'Boston Celtics',
       'Los Angeles Lakers', 'Cleveland Cavaliers', 'Sacramento Kings',
       'LA Clippers', 'Washington Wizards', 'Atlanta Hawks',
       'Toronto Raptors', 'New Orleans Pelicans', 'Orlando Magic',
       'Utah Jazz', 'Golden State Warriors', 'Chicago Bulls',
       'Miami Heat', 'Memphis Grizzlies', 'San Antonio Spurs',
       'Charlotte Hornets', 'Brooklyn Nets', 'Dallas Mavericks',
       'Houston Rockets', 'Philadelphia 76ers', 'Portland Trail Blazers',
       'Phoenix Suns', 'Denver Nuggets', 'Indiana Pacers',
       'Detroit Pistons', 'New York Knicks', 'Oklahoma City Thunder'],
      dtype=object)

In [273]:
data_2015["TEAM_ABBREVIATION"].unique()

array(['MIN', 'MIL', 'BOS', 'LAL', 'CLE', 'SAC', 'LAC', 'WAS', 'ATL',
       'TOR', 'NOP', 'ORL', 'UTA', 'GSW', 'CHI', 'MIA', 'MEM', 'SAS',
       'CHA', 'BKN', 'DAL', 'HOU', 'PHI', 'POR', 'PHX', 'DEN', 'IND',
       'DET', 'NYK', 'OKC'], dtype=object)

##### Isolate Cleveland and Warriors Stats

In [274]:
data_2015_cavs = data_2015[data_2015["TEAM_ABBREVIATION"] == "CLE"]
data_2015_wars = data_2015[data_2015["TEAM_ABBREVIATION"] == "GSW"]

#### Hypothesis Testing for `Points` (PTS)
If the Cavs and the Warriors offensive production stats are from the same distribution in the 2015/2016 seasons, then the distribution of points should be similar.

To do this, we will compare the two distributions against each other using the t-test, assuming that $$H_0: \mu_{cavs} = \mu_{wars}$$

We will using a pair-wise $t-test$, which is calculated as follows: $$t = \frac{\bar(x_1) - \bar(x_2)}{\sigma{\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}}$$

A large $t$-value will reveal that the difference between groups is larger than the pooled standard error, indicating a more significant difference between the groups. But first, is it safe to assume that the standard deviations between the two teams are equal?

##### Levene's Test for Equality of Variances, $H0: s_1 = s_2$
For this test, we will set the confidence level at $\alpha =$ 0.05

In [275]:
ttest, pval_pts = st.levene(data_2015_cavs["PTS"], data_2015_wars["PTS"])
print(f"The resulting p-value for Levene's test is {pval_pts:.2f} which is greater than the specified alpha value, 0.05.")
print("This suggests that the difference between the standard deviations of the two teams' PTS is insignificant.")

The resulting p-value for Levene's test is 0.89 which is greater than the specified alpha value, 0.05.
This suggests that the difference between the standard deviations of the two teams' PTS is insignificant.


##### Finding $\bar{x_1}$ and $\bar{x_2}$

In [276]:
# for the Cleveland Cavaliers, x1
cavs_2015_pts_mean = st.tmean(data_2015_cavs["PTS"])
# for the Golden State Warriors, x2
wars_2015_pts_mean = st.tmean(data_2015_wars["PTS"])

##### Finding $s_1$ and $s_2$ to determine $\sigma$ (*pooled standard variance*)

In [277]:
# for the Cleveland Cavaliers
cavs_2015_pts_std = st.tstd(data_2015_cavs["PTS"])
# for the Golden State Warriors
wars_2015_pts_std = st.tstd(data_2015_wars["PTS"])
# determine pooled standard variance
pts_sigma = st.tmean([cavs_2015_pts_std, wars_2015_pts_std])
pts_sigma

11.315425853726083

##### Finding $n_1$ and $n_2$

In [278]:
# for the Cleveland Cavaliers
cavs_2015_n = len(data_2015_cavs)
# for the Golden State Warriors
wars_2015_n = len(data_2015_wars)

##### Finding t-test values

In [279]:
tstat_from_calc = (cavs_2015_pts_mean - wars_2015_pts_mean) / (pts_sigma * np.sqrt(1/cavs_2015_n + 1/wars_2015_n))
tstat_from_calc

-5.976199196736252

##### Using *scipy* ttest

In [280]:
tstat_from_st, pvalue = st.ttest_ind(data_2015_cavs["PTS"], data_2015_wars["PTS"])

In [281]:
print(f"The t-statistic from the calculations: {tstat_from_calc:.2f}")
print(f"The t-statistic from the scipy module: {tstat_from_st:.2f}")

The t-statistic from the calculations: -5.98
The t-statistic from the scipy module: -5.97


In [282]:
print(f"The p-value rejects the null hypothesis that the points scored by the Cavs and the Warriors in the 2015/2016 season are from the same distribution.")

The p-value rejects the null hypothesis that the points scored by the Cavs and the Warriors in the 2015/2016 season are from the same distribution.


#### Hypothesis Testing for `Field Goal Percentage` (FG_PCT)

##### Levene's Test for Equality of Variances

In [283]:
_, pval_fgpct = st.levene(data_2015_cavs["FG_PCT"], data_2015_wars["FG_PCT"])
pval_fgpct
print(f"The resulting p-value for Levene's test is {pval_fgpct:.2f} which is greater than the specified alpha value, 0.05.")
print("This suggests that the difference between the standard deviations of the two teams' field goal percentages is insignificant.")

The resulting p-value for Levene's test is 0.42 which is greater than the specified alpha value, 0.05.
This suggests that the difference between the standard deviations of the two teams' field goal percentages is insignificant.


##### Finding t-test values

In [284]:
_, pvalue = st.ttest_ind(data_2015_cavs["FG_PCT"], data_2015_wars["FG_PCT"])
print(pvalue)

0.00206097581047554


---
### Task 3
Test the hypothesis that the number of points (PTS) scored by Cleveland Cavaliers changed significantly after the head coach changed in the 2015/2016 season.

- **Coach Blatt was fired on 24th of Jan, 2016**. 

Use the data from seasons 2014/2015 and 2015/2016 - those are years when Cleveland was coached by Blatt. 

**We have two possible solutions to try here:**
- Take the same amount of games from before and after and try t-test.
- Take all the games from before and after and look for the right test to compare two samples with different sizes. (You will need to go through the scipy documentation or google to figure out what kind of test is required.)

In [301]:
print(len(data_2014) + len(data_2015))

4920


In [298]:
data = pd.concat([data_2014, data_2015])

In [299]:
data

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
2460,22014,1610612760,OKC,Oklahoma City Thunder,21401222,2015-04-15,OKC @ MIN,W,241,138,...,0.710,18,28,46,22,10,6,13,25,25.0
2461,22014,1610612765,DET,Detroit Pistons,21401226,2015-04-15,DET @ NYK,W,240,112,...,0.741,11,33,44,20,14,3,12,22,22.0
2462,22014,1610612766,CHA,Charlotte Hornets,21401228,2015-04-15,CHA @ TOR,L,240,87,...,0.938,8,28,36,15,8,3,12,10,-5.0
2463,22014,1610612737,ATL,Atlanta Hawks,21401217,2015-04-15,ATL @ CHI,L,240,85,...,0.714,10,32,42,15,10,10,15,19,-6.0
2464,22014,1610612759,SAS,San Antonio Spurs,21401223,2015-04-15,SAS @ NOP,L,238,103,...,0.769,4,32,36,33,2,3,15,20,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22015,1610612744,GSW,Golden State Warriors,21500003,2015-10-27,GSW vs. NOP,W,241,111,...,0.909,21,35,56,29,8,7,20,29,16.0
2456,22015,1610612739,CLE,Cleveland Cavaliers,21500002,2015-10-27,CLE @ CHI,L,240,95,...,0.588,11,39,50,26,5,7,10,21,-2.0
2457,22015,1610612740,NOP,New Orleans Pelicans,21500003,2015-10-27,NOP @ GSW,L,241,95,...,0.704,8,25,33,21,9,3,18,26,-16.0
2458,22015,1610612741,CHI,Chicago Bulls,21500002,2015-10-27,CHI vs. CLE,W,240,97,...,0.696,7,40,47,13,6,10,13,22,2.0


In [285]:
data["GAME_DATE"]

0       2016-04-13
1       2016-04-13
2       2016-04-13
3       2016-04-13
4       2016-04-13
           ...    
7375    2013-10-29
7376    2013-10-29
7377    2013-10-29
7378    2013-10-29
7379    2013-10-29
Name: GAME_DATE, Length: 7380, dtype: object

#### Convert Dates to `datetime` Objects

In [286]:
for i, game_date in enumerate(data["GAME_DATE"]):
    data["GAME_DATE"][i] = dt.datetime.strptime(game_date, "%Y-%m-%d").date()

/var/folders/xm/yv4_9lnx4vxb435678j65d140000gn/T/ipykernel_23213/798130141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["GAME_DATE"][i] = dt.datetime.strptime(game_date, "%Y-%m-%d").date()


In [287]:
type(data["GAME_DATE"][0])

datetime.date

#### Isolate `Cleveland`

In [288]:
data_cle = data[data["TEAM_ABBREVIATION"] == "CLE"]

#### Use `GAME_DATE` to Divide Games from the

In [289]:
data_cle_after_blatt = data_cle[data_cle["GAME_DATE"] > dt.date(2016, 1, 24)]
data_cle_before_blatt = data_cle[data_cle["GAME_DATE"] <= dt.date(2016, 1, 24)]

In [290]:
data_cle_before_blatt.sort_values(by="GAME_DATE")

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7355,22013,1610612739,CLE,Cleveland Cavaliers,21300004,2013-10-30,CLE vs. BKN,W,241,98,...,0.676,16,32,48,21,7,3,11,20,4.0
7316,22013,1610612739,CLE,Cleveland Cavaliers,21300020,2013-11-01,CLE @ CHA,L,241,84,...,0.727,11,31,42,14,9,1,17,29,-6.0
7303,22013,1610612739,CLE,Cleveland Cavaliers,21300033,2013-11-02,CLE @ IND,L,240,74,...,0.867,6,31,37,11,15,1,16,25,-15.0
7282,22013,1610612739,CLE,Cleveland Cavaliers,21300047,2013-11-04,CLE vs. MIN,W,240,93,...,0.909,11,38,49,22,10,12,23,24,1.0
7248,22013,1610612739,CLE,Cleveland Cavaliers,21300066,2013-11-06,CLE @ MIL,L,240,104,...,0.727,12,27,39,20,10,2,9,21,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,22015,1610612739,CLE,Cleveland Cavaliers,21500601,2016-01-15,CLE @ HOU,W,241,91,...,0.652,17,32,49,19,10,6,14,24,14.0
1218,22015,1610612739,CLE,Cleveland Cavaliers,21500622,2016-01-18,CLE vs. GSW,L,240,98,...,0.680,8,28,36,22,3,4,15,25,-34.0
1195,22015,1610612739,CLE,Cleveland Cavaliers,21500631,2016-01-20,CLE @ BKN,W,240,91,...,0.700,6,39,45,15,6,3,13,13,13.0
1176,22015,1610612739,CLE,Cleveland Cavaliers,21500640,2016-01-21,CLE vs. LAC,W,239,115,...,0.750,14,33,47,29,5,2,13,31,13.0


#### Find `t-statistic` and `p-value`

In [ ]:
tstat_blatt, pval_blatt = st.ttest_ind(data_cle_before_blatt[""])

----------------


### Task 4
Download [**the similar dataset**](https://drive.google.com/file/d/0Bz9_0VdXvv9bRHhuRTI1aXBQcTA/view?usp=sharing&resourcekey=0-jGIaWyk0bXyECNSzB3x38w) with scores from playoff games in 2016.

------------
### Task 5
Test the hypothesis that **number of blocks (BLK)** are from the same distribution in both the NBA playoffs and in the NBA regular season for 2015/2016 seaon for the **Toronto Raptors**. 

- We will be working with two samples with different sizes again.


-----------------
### Task 6
Test the hypothesis that the number of points (PTS) scored by Cleveland Cavaliers is equally distributed for all 3 seasons. 

- In this case, we need a hypothesis test that compares more than 2 distributions at the same. (You will need to go through the scipy documentation or google to figure out what kind of test is required.)

#### Follow Up
**Between which seasons can we see the significant difference?**

+ Unfortunatelly, this is not the output of an ANOVA test and further tests are needed to be run.
+ Note: Lebron James came back to the Cleveland Caveliers prior to season 2014/2015. We can use this fact to interpret our results correctly.